In [1]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import math
import torch
from copy import deepcopy
from itertools import chain 
from torch.utils.data import Dataset
from torchtext.vocab import vocab as Vocab

import warnings
warnings.filterwarnings("ignore")


Pathing


In [2]:
#Lokalt
data_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\data'

In [ ]:
#saga
data_dir = "/home/aeerik/data/raw/"


CONFIG

In [41]:
# Budget config file
include_pheno = False
threshold_year = 1970
path = data_dir
max_length = 51
mask_prob = 0.15
embedding_dim = 32
drop_prob = 0.2

#Encoder
enc_dim_inp = 32 
enc_dim_out = 32 
attention_heads = 8 

#BERT
num_encoders = 2

#trainer
epochs = 5
batch_size = 32
lr = 0.001
stop_patience = 10


Data import

In [5]:
from data_preprocessing import data_loader
include_pheno = False
threshold_year = 1970
path = data_dir

NCBI = data_loader(include_pheno,threshold_year,path)
NCBI.head()

,year,location,genes
1,[PAD],USA,"[aadA1, sul1, tet(A)]"
4,[PAD],USA,"[sul2, aph(3'')-Ib, tet(A), aph(6)-Id]"
5,[PAD],USA,"[glpT_E448K=POINT, pmrB_Y358N=POINT]"
6,[PAD],Sweden,"[cyaA_S352T=POINT, uhpT_E350Q=POINT, glpT_E448..."
7,1979,USA,[glpT_E448K=POINT]


Vocabulary

In [6]:
from build_vocabulary import make_vocabulary
vocabulary = make_vocabulary(NCBI)
print(len(vocabulary))

1208


In [9]:
from create_dataset import NCBIDataset
from build_vocabulary import make_vocabulary
from data_preprocessing import data_loader

include_pheno = False
threshold_year = 1970
path = data_dir

NCBI = data_loader(include_pheno,threshold_year,path)

max_length = 51
mask_prob = 0.50
vocabulary = make_vocabulary(NCBI)

test_set = NCBIDataset(NCBI, vocabulary, max_length, mask_prob)
test_set.prepare_dataset()

In [31]:
print(test_set[1][1])
token_mask = test_set[1][1] != -1
print(token_mask)

masked_idx = torch.masked_select(test_set[1][1], token_mask)
print(masked_idx)

tensor([ -1,  -1,  -1, 216, 217,  -1, 215,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1])
tensor([False, False, False,  True,  True, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False, False])
tensor([216, 217, 215])


In [17]:
from torch.utils.data import Subset
copy_dataset = Subset(test_set, range(len(test_set)))

In [19]:
copy_dataset[1]


TypeError: range indices must be integers or slices, not str

Embedding 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class JointEmbedding(nn.Module):

    def __init__(self, embedding_dim, vocab_size, max_length, drop_prob):
        super(JointEmbedding, self).__init__()

        self.max_length = max_length
        self.drop_prob = drop_prob
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.token_emb = nn.Embedding(self.vocab_size, self.embedding_dim)

        self.dropout = nn.Dropout(drop_prob)	
        self.norm = nn.LayerNorm(self.embedding_dim)

    def forward(self, input_tensor):
        token_embedding = self.token_emb(input_tensor)
        token_embedding = self.norm(token_embedding)
        token_embedding = self.dropout(token_embedding)

        return token_embedding


In [10]:
from embedding import JointEmbedding
embedding_dim = 32
voca_size = len(vocabulary)
max_length = 20
drop_prob = 0.2

emb_test = JointEmbedding(embedding_dim, voca_size, max_length, drop_prob)
emb_test.forward(test_set[1][0])

tensor([[-1.0570,  0.4458,  0.2012,  ..., -3.8613,  0.1622,  0.9274],
        [-0.6363,  2.4782,  1.9085,  ..., -0.0000, -3.4922, -0.0000],
        [-0.0058, -0.0000, -1.1447,  ...,  1.5867, -0.0000,  2.2315],
        ...,
        [-0.6363,  2.4782,  0.0000,  ..., -1.4776, -3.4922, -1.0516],
        [-0.0000,  2.4782,  0.0000,  ..., -1.4776, -3.4922, -0.0000],
        [-0.0000,  2.4782,  1.9085,  ..., -1.4776, -3.4922, -1.0516]],
       grad_fn=<MulBackward0>)

In [ ]:
import torch

from torch import nn
import torch.nn.functional as f

class AttentionHead(nn.Module):

    def __init__(self, dim_inp, dim_out, drop_prob):
        super(AttentionHead, self).__init__()

        self.dim_inp = dim_inp
        self.drop_prob = drop_prob
        self.dropout = nn.Dropout(drop_prob)
        self.q = nn.Linear(dim_inp, dim_out)
        self.k = nn.Linear(dim_inp, dim_out)
        self.v = nn.Linear(dim_inp, dim_out)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor = None):
        query, key, value = self.q(input_tensor), self.k(input_tensor), self.v(input_tensor)

        scale = query.size(1) ** 0.5
        scores = torch.matmul(query, key.transpose(-1, -2)) / scale

        scores = scores.masked_fill_(attention_mask, -1e9)
        attn = f.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        context = torch.matmul(attn, value)

        return context


class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, dim_inp, dim_out,drop_prob):
        super(MultiHeadAttention, self).__init__()

        self.heads = nn.ModuleList([
            AttentionHead(dim_inp, dim_out,drop_prob) for _ in range(num_heads)
        ])
        self.linear = nn.Linear(dim_out * num_heads, dim_inp)
        self.norm = nn.LayerNorm(dim_inp)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor):
        s = [head(input_tensor, attention_mask) for head in self.heads]
        scores = torch.cat(s, dim=-1)
        scores = self.linear(scores)
        return self.norm(scores)
    
attention_test = AttentionHead(32, 32, 0.2)
attention_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

multihead_test = MultiHeadAttention(8, 32, 32,0.2)
multihead_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

In [ ]:
import torch

from torch import nn
import torch.nn.functional as f

#dropout 
#num heads
#dim in, dim out

class Encoder(nn.Module):

    def __init__(self, dim_inp, dim_out, attention_heads, dropout_prob):
        super(Encoder, self).__init__()
        self.dropout_prob = dropout_prob
        self.attention_heads = attention_heads
        self.dim_inp = dim_inp
        self.dim_out = dim_out
        
        
        self.attention = MultiHeadAttention(self.attention_heads, self.dim_inp, self.dim_out, self.dropout_prob)  # batch_size x sentence size x dim_inp
        self.feed_forward = nn.Sequential(
            nn.Linear(self.dim_inp, self.dim_out),
            nn.Dropout(self.dropout_prob),
            nn.GELU(),
            nn.Linear(self.dim_out, self.dim_inp),
            nn.Dropout(self.dropout_prob)
        )
        self.norm = nn.LayerNorm(self.dim_inp)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor):
        context = self.attention(input_tensor, attention_mask)
        res = self.feed_forward(context)
        return self.norm(res)

encoder_test = Encoder(enc_dim_inp, enc_dim_out, attention_heads, drop_prob)
encoder_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

In [ ]:
class BERT(nn.Module):

    def __init__(self, vocab_size, max_length, dim_inp, dim_out, attention_heads, num_encoders, dropout_prob):
        super(BERT, self).__init__()
        self.attention_heads = attention_heads
        self.max_length = max_length
        self.vocab_size = vocab_size
        self.dim_inp = dim_inp  
        self.dim_out = dim_out
        self.num_encoders = num_encoders
        self.dropout_prob = dropout_prob  

        self.embedding = JointEmbedding(self.dim_inp, self.vocab_size, self.max_length, self.dropout_prob)
        self.encoders = nn.ModuleList([Encoder(self.dim_inp, self.dim_out, self.attention_heads, self.dropout_prob) for _ in range(self.num_encoders)])

        self.token_prediction_layer = nn.Linear(self.dim_inp, self.vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, input_tensor: torch.Tensor, attention_mask: torch.Tensor):
        embedded = self.embedding(input_tensor)
        for layer in self.encoders:
            embedded = layer(embedded, attention_mask)

        token_predictions = self.token_prediction_layer(embedded)
        return self.softmax(token_predictions)

In [ ]:
bert_test = BERT(len(vocabulary), 20, 32, 32, 8, 2, 0.2)
bert_test.forward(test_set[1][0], test_set[1][2])

In [ ]:
from bert_builder import AttentionHead

attention_test = AttentionHead(32, 32, 0.2)
attention_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

In [ ]:
from bert_builder import MultiHeadAttention
multihead_test = MultiHeadAttention(num_heads=8, dim_inp=32, dim_out=32, drop_prob=0.2)
multihead_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

In [ ]:
from bert_builder import Encoder

encoder_test = Encoder(32, 32, 8, 0.2)
encoder_test.forward(emb_test.forward(test_set[1][0]), test_set[1][2])

In [14]:
from bert_builder import BERT

bert_test = BERT(vocab_size=len(vocabulary), max_length=51, dim_inp=32, dim_out=32, attention_heads=8, num_encoders=2, dropout_prob=0.2)
bert_test.forward(test_set[1][0], test_set[1][2]).size()

torch.Size([51, 1208])

---------------------------

Trainer

In [5]:
import numpy as np
from create_dataset import NCBIDataset
def get_split_indices(size_to_split, val_share, random_state: int = 42):
    indices = np.arange(size_to_split)
    np.random.seed(random_state)
    np.random.shuffle(indices)
    
    train_share = 1 - val_share
    
    train_size = int(train_share * size_to_split)
    
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]
    
    return train_indices, val_indices
max_length = 20
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary, max_length, mask_prob)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary, max_length, mask_prob)

In [51]:
import time
from datetime import datetime
from torch import nn
from torch.utils.data import DataLoader


class BertTrainer:
    def __init__(self, model, train_set, val_set, epochs, batch_size, lr, device, stop_patience, results_dir):
        
        self.model = model
        self.train_set = train_set
        self.train_size = len(train_set)
        self.val_set = val_set
        self.epochs = epochs    
        self.batch_size = batch_size
        self.num_batches = self.train_size // self.batch_size
        self.lr = lr
        self.weight_decay = 0.01
        self.current_epoch  = 0
        self.early_stopping_counter = 0	
        self.patience = stop_patience
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        self.criterion = nn.CrossEntropyLoss(ignore_index = -1).to(device)

        self.device = device
        self.results_dir = results_dir


    def __call__(self):      
        self.val_set.prepare_dataset() 
        self.val_loader = DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False)
        start_time = time.time()
        self.best_val_loss = float('inf')
        self._init_result_lists()
        for self.current_epoch in range(self.current_epoch, self.epochs):
            #Training
            self.model.train()
            self.train_set.prepare_dataset()
            self.train_loader = DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)
            epoch_start_time = time.time()
            avg_epoch_loss = self.train(self.current_epoch)
            self.losses.append(avg_epoch_loss) 
            print(f"Epoch completed in {(time.time() - epoch_start_time)/60:.1f} min")
            
            #Validation
            print("Evaluating on validation set...")
            val_results = self.evaluate(self.val_loader, self.val_set)
            print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time))}")
            self.val_losses.append(val_results[0])  
            self.val_accs.append(val_results[1])

            criterion = self.stop_early()
        print(f"-=Training completed=-")
        results = {
            "best_epoch": self.current_epoch,
            "train_losses": self.losses,
            "val_losses": self.val_losses,
            "val_accs": self.val_accs
        }
        return results

    def _init_result_lists(self):
        self.losses = []
        self.val_losses = []
        self.val_accs = []
    
    def stop_early(self):
        if self.val_losses[-1] < self.best_val_loss:
            self.best_val_loss = self.val_losses[-1]
            self.best_epoch = self.current_epoch
            self.best_model_state = self.model.state_dict()
            self.early_stopping_counter = 0
            return False
        else:
            self.early_stopping_counter += 1
            return True if self.early_stopping_counter >= self.patience else False

    def train(self, epoch: int):
        print(f"Epoch {epoch+1}/{self.epochs}")
        time_ref = time.time()
        
        epoch_loss = 0
        reporting_loss = 0
        printing_loss = 0
        for i, batch in enumerate(self.train_loader):
            input, token_target, attn_mask = batch
            
            self.optimizer.zero_grad() 

            tokens = self.model(input, attn_mask) 
            loss = self.criterion(tokens.transpose(-1, -2), token_target) 
            
            epoch_loss += loss.item() 
            reporting_loss += loss.item()
            printing_loss += loss.item()
            
            loss.backward() 
            self.optimizer.step()         
        avg_epoch_loss = epoch_loss / self.num_batches
        return avg_epoch_loss 
    
    def evaluate(self, loader, dataset):
        self.model.eval()
        epoch_loss = 0
        total_correct = 0
        total_tokens = 0

        with torch.no_grad():
            for i, batch in enumerate(loader):
                input, token_target, attn_mask = batch
                tokens = self.model(input, attn_mask)
                loss = self.criterion(tokens.transpose(-1, -2), token_target)
                epoch_loss += loss.item()
                
                token_mask =  token_target != -1
                predicted_tokens = tokens.argmax(dim=-1)
                token_target = torch.masked_select(token_target, token_mask)
                predicted_tokens = torch.masked_select(predicted_tokens, token_mask)
                
                correct = (predicted_tokens == token_target).sum().item()
                total_correct += correct
                total_tokens += token_target.numel() 
        
        avg_epoch_loss = epoch_loss / len(loader)
        accuracy = total_correct / total_tokens

        return avg_epoch_loss, accuracy
    
    def _save_model(self, savepath: Path):
        torch.save(self.best_model_state, savepath)
        print(f"Model saved to {savepath}")
        
        
    def _load_model(self, savepath: Path):
        print(f"Loading model from {savepath}")
        self.model.load_state_dict(torch.load(savepath))
        print("Model loaded")

    

In [52]:
from bert_builder import BERT
from data_preprocessing import data_loader
from build_vocabulary import make_vocabulary
from misc import get_split_indices
from create_dataset import NCBIDataset


NCBI = data_loader(include_pheno,threshold_year,path)
vocabulary = make_vocabulary(NCBI)
reduced_samples = 1000
NCBI = NCBI.head(reduced_samples)
print(f"Data loaded, {len(NCBI)} samples found")
print(f"length  of vocabulary:",len(vocabulary))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary, max_length, mask_prob)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary, max_length, mask_prob)

bert_test = BERT(len(vocabulary), max_length, enc_dim_inp, enc_dim_out, attention_heads, num_encoders, drop_prob)

test = BertTrainer(bert_test, train_set, val_set, epochs, batch_size, lr, device, stop_patience, data_dir)
test()
test._save_model('c:\\Users\\erika\\Desktop\\Exjobb\\test_model.pth')

Data loaded, 1000 samples found
length  of vocabulary: 1208
Epoch 1/5
Epoch completed in 0.0 min
Evaluating on validation set...
Elapsed time: 00:00:03
Epoch 2/5
Epoch completed in 0.0 min
Evaluating on validation set...
Elapsed time: 00:00:06
Epoch 3/5
Epoch completed in 0.0 min
Evaluating on validation set...
Elapsed time: 00:00:09
Epoch 4/5
Epoch completed in 0.0 min
Evaluating on validation set...
Elapsed time: 00:00:12
Epoch 5/5
Epoch completed in 0.0 min
Evaluating on validation set...
Elapsed time: 00:00:16
Training completed
Model saved to c:\Users\erika\Desktop\Exjobb\test_model.pth


In [53]:
test._load_model('c:\\Users\\erika\\Desktop\\Exjobb\\test_model.pth')

Loading model from c:\Users\erika\Desktop\Exjobb\test_model.pth
Model loaded
